In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from encode_data import *
from midi_data import *

In [3]:
from tqdm import tqdm

## Standardize and reformat raw midi files before encoding to text
- Transform key to C major
- Remove unused instruments
- Combine multiple tracks with the same instrument into a single part
- Melody, Piano, String

### Load midi data

In [4]:
version = 'v7'
data_path = Path('data/midi')
version_path = data_path/version

In [5]:
import pandas as pd

In [6]:
source_dir = 'midi_sources'
out_dir = 'midi_transform'
source_csv = version_path/'metadata'/f'{source_dir}.csv'
out_csv = version_path/out_dir/f'{out_dir}.csv'
out_csv.parent.mkdir(parents=True, exist_ok=True)
source_csv, out_csv

(PosixPath('data/midi/v7/metadata/midi_sources.csv'),
 PosixPath('data/midi/v7/midi_transform/midi_transform.csv'))

In [7]:
df = pd.read_csv(source_csv); df.head()

/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (0,9,16,17,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,song_url,instruments,ht_offset,time_signature,title,quarter_length,ht_mode,source,artist,ht_key,...,midi,inferred_key,inferred_offset,genres,section,parts,ht_time_signature,ht_bpm,midi_title,mxl
0,https://www.hooktheory.com/theorytab/view/wayn...,Piano,0.0,3/4,yu-gi-oh-theme-song,36.0,1.0,hooktheory,wayne-sharpe,C,...,midi_sources/hooktheory/pianoroll/w/wayne-shar...,C major,0.0,NaN,intro,"intro,chorus",3.0,85.0,yu-gi-oh,NaN
1,https://www.hooktheory.com/theorytab/view/wayn...,"Piano,Piano",0.0,4/4,yu-gi-oh-theme-song,32.0,1.0,hooktheory,wayne-sharpe,C,...,midi_sources/hooktheory/pianoroll/w/wayne-shar...,C major,0.0,NaN,chorus,"intro,chorus",4.0,128.0,yu-gi-oh3,NaN
2,https://www.hooktheory.com/theorytab/view/what...,"Piano,Piano",-5.0,4/4,kiefer,16.0,6.0,hooktheory,what-a-day,D,...,midi_sources/hooktheory/pianoroll/w/what-a-day...,A minor,0.0,Jazz,chorus,chorus,4.0,96.0,kiefer,NaN
3,https://www.hooktheory.com/theorytab/view/whit...,"Piano,Piano",-5.0,4/4,senbonzakura,32.0,6.0,hooktheory,whiteflame,D,...,midi_sources/hooktheory/pianoroll/w/whiteflame...,D minor,-5.0,"J-Pop,Pop",verse,"verse,pre-chorus,chorus",4.0,152.0,Senbonzakura,NaN
4,https://www.hooktheory.com/theorytab/view/wham...,"Piano,Piano",-1.0,4/4,last-christmas,32.0,1.0,hooktheory,wham,Db,...,midi_sources/hooktheory/pianoroll/w/wham/last-...,C# major,-1.0,Holiday,intro,"intro,verse,chorus",4.0,108.0,Last Christmas Intro,NaN


In [8]:
df_filtered = df.loc[df.time_signature.notna()]

In [9]:
from data_sources import process_parallel, transform_csv_row

In [10]:
from midi_transform import *

### Need better midi formatting

In [11]:
# v6
# def transform_func(file, out_file, row):
#     offset = row.ht_offset if row.source == 'hooktheory' else row.inferred_offset
#     transform_midi(file, out_file, offset=offset)

In [12]:
def transform_func(file, out_file, row):
    offset = row.ht_offset if row.source == 'hooktheory' else row.inferred_offset
    music_file = compress_midi_file(file, cutoff=8) # remove non note tracks and standardize instruments
    music_file.open(out_file, attrib='wb')
    music_file.write()
    music_file.close()

In [13]:
from functools import partial
parallel_func = partial(transform_csv_row, 
        transform_func=transform_func,
        base_path=version_path,
        source_dir=source_dir,
        out_dir=out_dir,
        out_extension=None,
        source_col='midi'
       )

In [14]:
# sanity check
# for r in df_filtered.iterrows():
#     parallel_func(r)

In [15]:
### AS TOD: Fix 'info channel is not channel 0' error

In [16]:
idx2out = process_parallel(parallel_func, df_filtered.iterrows(), total=df_filtered.shape[0])

Error converting midi to sequence 'NoneType' object has no attribute 'open'
Error converting midi to sequence 'NoneType' object has no attribute 'open'
Error converting midi to sequence 'NoneType' object has no attribute 'open'
Error converting midi to sequence 'NoneType' object has no attribute 'open'


midi: WARNING: Conversion error for <MidiEvent None, t=None, track=0, channel=None>: unknown midi event type: None; ignored.


Error converting midi to sequence 'NoneType' object has no attribute 'open'
Error converting midi to sequence 'NoneType' object has no attribute 'open'
Error converting midi to sequence 'NoneType' object has no attribute 'open'
Error converting midi to sequence 'NoneType' object has no attribute 'open'
Error converting midi to sequence 'NoneType' object has no attribute 'open'
Error converting midi to sequence 'NoneType' object has no attribute 'open'
Error converting midi to sequence 'NoneType' object has no attribute 'open'
Error converting midi to sequence 'NoneType' object has no attribute 'open'


midi: WARNING: Conversion error for <MidiEvent None, t=None, track=8, channel=None>: unknown midi event type: None; ignored.
midi: WARNING: Conversion error for <MidiEvent None, t=None, track=9, channel=None>: unknown midi event type: None; ignored.


Error converting midi to sequence 'NoneType' object has no attribute 'open'
Error converting midi to sequence 'NoneType' object has no attribute 'open'
Error converting midi to sequence 'NoneType' object has no attribute 'open'
Error converting midi to sequence 'NoneType' object has no attribute 'open'
Error converting midi to sequence 'NoneType' object has no attribute 'open'
Error converting midi to sequence 'NoneType' object has no attribute 'open'
Error converting midi to sequence 'NoneType' object has no attribute 'open'
Error converting midi to sequence 'NoneType' object has no attribute 'open'
Error converting midi to sequence 'NoneType' object has no attribute 'open'
Error converting midi to sequence 'NoneType' object has no attribute 'open'
Error converting midi to sequence 'NoneType' object has no attribute 'open'
Error converting midi to sequence 'NoneType' object has no attribute 'open'
Error converting midi to sequence 'NoneType' object has no attribute 'open'
Error conver

...

midi: WARNING: Conversion error for <MidiEvent None, t=None, track=0, channel=None>: unknown midi event type: None; ignored.


Error converting midi to sequence 'NoneType' object has no attribute 'open'
Error converting midi to sequence 'NoneType' object has no attribute 'open'
Error converting midi to sequence 'NoneType' object has no attribute 'open'
Error converting midi to sequence 'NoneType' object has no attribute 'open'
Error converting midi to sequence 'NoneType' object has no attribute 'open'
Error converting midi to sequence 'NoneType' object has no attribute 'open'
Error converting midi to sequence 'NoneType' object has no attribute 'open'
Error converting midi to sequence 'NoneType' object has no attribute 'open'
Error converting midi to sequence 'NoneType' object has no attribute 'open'
Error converting midi to sequence 'NoneType' object has no attribute 'open'
Error converting midi to sequence 'NoneType' object has no attribute 'open'
Error converting midi to sequence 'NoneType' object has no attribute 'open'
Error converting midi to sequence 'NoneType' object has no attribute 'open'
Error conver

midi: WARNING: Conversion error for <MidiEvent None, t=None, track=2, channel=None>: unknown midi event type: None; ignored.
midi: WARNING: Conversion error for <MidiEvent None, t=None, track=6, channel=None>: unknown midi event type: None; ignored.


Error converting midi to sequence 'NoneType' object has no attribute 'open'
Error converting midi to sequence 'NoneType' object has no attribute 'open'
Error converting midi to sequence 'NoneType' object has no attribute 'open'
Error converting midi to sequence 'NoneType' object has no attribute 'open'
Error converting midi to sequence 'NoneType' object has no attribute 'open'
Error converting midi to sequence 'NoneType' object has no attribute 'open'
Error converting midi to sequence 'NoneType' object has no attribute 'open'
Error converting midi to sequence 'NoneType' object has no attribute 'open'
Error converting midi to sequence 'NoneType' object has no attribute 'open'
Error converting midi to sequence 'NoneType' object has no attribute 'open'
Error converting midi to sequence 'NoneType' object has no attribute 'open'
Error converting midi to sequence 'NoneType' object has no attribute 'open'
Error converting midi to sequence 'NoneType' object has no attribute 'open'
Error conver

Error converting midi to sequence badly formated midi bytes, got: b''
Error converting midi to sequence badly formated midi bytes, got: b''
Error converting midi to sequence badly formated midi bytes, got: b''
Error converting midi to sequence badly formated midi bytes, got: b''
Error converting midi to sequence badly formated midi bytes, got: b''
Error converting midi to sequence badly formated midi bytes, got: b''
Error converting midi to sequence badly formated midi bytes, got: b''
Error converting midi to sequence badly formated midi bytes, got: b''
Error converting midi to sequence badly formated midi bytes, got: b''
Error converting midi to sequence badly formated midi bytes, got: b''
Error converting midi to sequence badly formated midi bytes, got: b''
Error converting midi to sequence badly formated midi bytes, got: b''
Error converting midi to sequence badly formated midi bytes, got: b''
Error converting midi to sequence badly formated midi bytes, got: b''
Error converting mid

In [17]:
tdf = pd.DataFrame(data={out_dir: list(idx2out.values())}, index=list(idx2out.keys()))

In [18]:
merged_df = df.join(tdf)

In [19]:
tdf.shape, df.shape, merged_df.shape

((39217, 1), (39942, 22), (39942, 23))

In [20]:
merged_df.to_csv(out_csv, index=False); merged_df.head()

,song_url,instruments,ht_offset,time_signature,title,quarter_length,ht_mode,source,artist,ht_key,...,inferred_key,inferred_offset,genres,section,parts,ht_time_signature,ht_bpm,midi_title,mxl,midi_transform
0,https://www.hooktheory.com/theorytab/view/wayn...,Piano,0.0,3/4,yu-gi-oh-theme-song,36.0,1.0,hooktheory,wayne-sharpe,C,...,C major,0.0,NaN,intro,"intro,chorus",3.0,85.0,yu-gi-oh,NaN,midi_transform/hooktheory/pianoroll/w/wayne-sh...
1,https://www.hooktheory.com/theorytab/view/wayn...,"Piano,Piano",0.0,4/4,yu-gi-oh-theme-song,32.0,1.0,hooktheory,wayne-sharpe,C,...,C major,0.0,NaN,chorus,"intro,chorus",4.0,128.0,yu-gi-oh3,NaN,midi_transform/hooktheory/pianoroll/w/wayne-sh...
2,https://www.hooktheory.com/theorytab/view/what...,"Piano,Piano",-5.0,4/4,kiefer,16.0,6.0,hooktheory,what-a-day,D,...,A minor,0.0,Jazz,chorus,chorus,4.0,96.0,kiefer,NaN,midi_transform/hooktheory/pianoroll/w/what-a-d...
3,https://www.hooktheory.com/theorytab/view/whit...,"Piano,Piano",-5.0,4/4,senbonzakura,32.0,6.0,hooktheory,whiteflame,D,...,D minor,-5.0,"J-Pop,Pop",verse,"verse,pre-chorus,chorus",4.0,152.0,Senbonzakura,NaN,midi_transform/hooktheory/pianoroll/w/whitefla...
4,https://www.hooktheory.com/theorytab/view/wham...,"Piano,Piano",-1.0,4/4,last-christmas,32.0,1.0,hooktheory,wham,Db,...,C# major,-1.0,Holiday,intro,"intro,verse,chorus",4.0,108.0,Last Christmas Intro,NaN,midi_transform/hooktheory/pianoroll/w/wham/las...
